# WNTR Additional Examples

Here, we will have a more advanced example, using WNTR to run a stochastic simulation


## Imports
Install and import WNTR and additional Python packages that are needed for the tutorial
- Numpy is required to define comparison operators (i.e., np.greater) in queries
- Matplotlib is required to create graphics

In [ ]:
# Install required packages if not already available
try:
    import wntr
except ImportError:
    !pip install wntr
    import wntr  # import again after installation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Units
WNTR uses **SI (International System) units (length in meters, time in seconds, mass in kilograms)**.  See https://usepa.github.io/WNTR/units.html for more details.

# Running a stochastic simulation
Often, system conditions, such as consumer demands are dynamic and uncertain, requiring us to update simulations to account for these changes.
In this example, we demonstrate how to model such uncertainty by randomly varying the demands and examining how this affects the tank water level.

Main steps:
1. Select a node of interest.
2. Introduce random uncertainty in the demand, assuming a uniform distribution between 0.5 and 1.5 of the base demand.
3. Run hydraulic simulations for both the original and modified demands.
4. Plot the node demands and pressures, the tank water level, and the pump flow for comparison between the two conditions.

## Get node, tank, pump and run basic simulation

In [ ]:
# Import .inp and run simulation
inp = '/content/Net3.inp'
wn = wntr.network.WaterNetworkModel(inp)
sim = wntr.sim.EpanetSimulator(wn)
results_base = sim.run_sim() # this is the step that actually runs the simulation

units = 15850.3 # cms to gpm

In [ ]:
# get all node names
node_names = wn.junction_name_list
print(node_names)

In [ ]:
# let's use junction 109 as an example
idx = node_names.index('109') # find the index of '109'
idx

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# select this node
node = wn.get_node(node_names[idx])
node

In [ ]:
node.demand_timeseries_list[0].pattern_name

In [ ]:
# store node ID
selected_node = node.name
print(selected_node)

In [ ]:
# get tank list
print("Tank names", wn.tank_name_list)

# get pump list
print("Pump names", wn.pump_name_list)

In [ ]:
# let's use tank '1' and pump '10'
tank = wn.tank_name_list[0]
tank

In [ ]:
pump = wn.pump_name_list[0]
pump

In [ ]:
# get base results

# base results: pressure, demand, tank, pump
results_demand_base = results_base.node['demand'][selected_node]*units # we use results_base because that is the name we gave to our results variable
results_pressure_base = results_base.node['pressure'][selected_node]
results_tank_base = results_base.node['pressure'][tank]
results_pump_base = results_base.link['flowrate'][pump]*units
time_hours = results_pressure_base.index / 3600  # Convert seconds to hours


In [ ]:
results_demand_base

## Update demand, run new simulation, and compare results

In [ ]:
# get base demand
base_demand = node.demand_timeseries_list[0].base_value
print(base_demand)

In [ ]:
# generate random number between 0.5 to 1.5
multiplier = np.random.uniform(0.5, 1.5)
print(multiplier)

In [ ]:
# update demand
node.demand_timeseries_list[0].base_value = base_demand*multiplier
print(node.demand_timeseries_list[0].base_value)

In [ ]:
# run new simulation
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()

In [ ]:
# get new results

# new results: pressure, demand, tank, pump
results_demand_new = results.node['demand'][selected_node]*units
results_pressure_new = results.node['pressure'][selected_node]
results_tank_new = results.node['pressure'][tank]
results_pump_new = results.link['flowrate'][pump]*units


In [ ]:
results_demand_base.head()

In [ ]:
results_demand_new.head()

In [ ]:
# Make some plots

# demand
plt.plot(time_hours, results_demand_base, color='gray', linewidth = 1, alpha = 0.5)
plt.plot(time_hours, results_demand_new, color='blue', linewidth = 1, alpha = 0.5)
# Formatting the plot
plt.xlabel('Time (hours)')
plt.ylabel('Demand (gpm)')
plt.title('Demand at node ' + selected_node)
plt.legend(['base','new'], loc='best') # labels are assigned based on the order that you plot them
plt.show()


In [ ]:
# pressure
plt.plot(time_hours, results_pressure_base, color='gray', linewidth = 1, alpha = 0.5)
plt.plot(time_hours, results_pressure_new, color='blue', linewidth = 1, alpha = 0.5)
# Formatting the plot
plt.xlabel('Time (hours)')
plt.ylabel('Pressure head (m)')
plt.title('Pressure at node ' + selected_node)
plt.legend({'base','new'}, loc='best')
plt.show()

In [ ]:
# tank
plt.plot(time_hours, results_tank_base, color='gray', linewidth = 1, alpha = 0.5)
plt.plot(time_hours, results_tank_new, color='blue', linewidth = 1, alpha = 0.5)
# Formatting the plot
plt.xlabel('Time (hours)')
plt.ylabel('Water level (m)')
plt.title('Water level tank ' + tank)
plt.legend({'base','new'}, loc='best')
plt.show()

In [ ]:
# pump
plt.plot(time_hours, results_pump_base, color='gray', linewidth = 1, alpha = 0.5)
plt.plot(time_hours, results_pump_new, color='blue', linewidth = 1, alpha = 0.5)
# Formatting the plot
plt.xlabel('Time (hours)')
plt.ylabel('Flow (gpm)')
plt.title('Pump flow ' + pump)
plt.legend({'base','new'}, loc='best')
plt.show()

**Additional notes:**
Next, you can use the loops we covered last time, change demands at all nodes, and see the impact of uncertainty in demand